In [2]:
import pandas as pd

# 1.1 import first dataset: university rankings
urank = pd.read_csv("https://raw.githubusercontent.com/caifand/DMRL_THA/master/FinalPaper/0_data_preprocessing/0_raw/cwur.csv")

# check the import
print(urank.head(10))
print(urank.tail(10))
print(list(urank))
print(urank.shape)

   World Rank                            Institution Location  National Rank  \
0           1                     Harvard University      USA              1   
1           2                    Stanford University      USA              2   
2           3  Massachusetts Institute of Technology      USA              3   
3           6      University of California Berkeley      USA              4   
4           7                   Princeton University      USA              5   
5           8                    Columbia University      USA              6   
6           9     California Institute of Technology      USA              7   
7          10                  University of Chicago      USA              8   
8          11                        Yale University      USA              9   
9          13             University of Pennsylvania      USA             10   

   Quality of Education Alumni Employment  Quality of Faculty Research Output  \
0                   2.0               

In [3]:
# 1.2 drop needless columns
cite_rank = urank[['Institution', 'Citations']]

# rename columns
cite_rank.rename(columns={'Institution':'institution',
                    'Citations':'citation'}, 
                 inplace=True)

print(cite_rank.shape)
print(cite_rank.head(5))

(213, 2)
                             institution citation
0                     Harvard University        1
1                    Stanford University        2
2  Massachusetts Institute of Technology        6
3      University of California Berkeley        8
4                   Princeton University       27


c:\users\d\appdata\local\programs\python\python37\lib\site-packages\pandas\core\frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [4]:
# 1.3 check null values and data types
cite_rank.isna().sum()
cite_rank.dtypes

institution    object
citation       object
dtype: object

In [5]:
# transform data types
#cite_rank['Citations'] = cite_rank['Citations'].astype('int64')
#cite_rank.dtypes

# First I got error message: "ValueError: invalid literal for int() with base 10: '> 1000'"
# This is because some observations have the value of citation indicators as "> 1000", which can not be converted to integer data type

# normalize column values using replace method
cite_rank = cite_rank.replace('> 1000', '1000')

In [6]:
# trasform data type
cite_rank['citation'] = cite_rank['citation'].astype('int64')
cite_rank.dtypes

institution    object
citation        int64
dtype: object

In [7]:
# 2.1 import the second dataset: open access repositories
oa_repo = pd.read_csv('https://raw.githubusercontent.com/caifand/DMRL_THA/master/FinalPaper/0_data_preprocessing/0_raw/doar.csv')

print(oa_repo.head(5))
print(oa_repo.shape)

   repositoryid                              preferred_name  \
0           382  11th Joint Symposium on Neural Computation   
1          3428          ACHS, Theses and Capstone Projects   
2          4464                                        ARCH   
3          2590                      ASU Digital Repository   
4          3015                                      AUrora   

                                           name_name  \
0         11th Joint Symposium on Neural Computation   
1  American College of Healthcare Sciences, These...   
2                                               ARCH   
3                             ASU Digital Repository   
4             Auburn University Scholarly Repository   

                         name_acronym name_preferred name_language  \
0                                 NaN           name            en   
1  ACHS, Theses and Capstone Projects        acronym            en   
2                                 NaN           name            en   
3   

In [8]:
# 2.2 truncate needless columns

# check columns and missing values
oa_repo.isna().sum()

repositoryid                      0
preferred_name                    0
name_name                         0
name_acronym                    463
name_preferred                    2
name_language                     2
identifiers_type                373
identifiers_identifier          373
type                              0
status                            0
repository_status                 0
description                       2
public_notes                    433
private_notes                   332
publicly_visible                  0
repository_url                    0
oai_url                         281
software_name                    88
software_version                455
software_name_other             375
year_established                503
content_languages                 1
content_types                     0
content_subjects                  0
metadata_record_count            81
full_text_record_count          373
count_source                    427
preferred_org_name          

In [9]:
# only keep needed columns
oa_repo = oa_repo[['preferred_name', 'year_established', 'preferred_org_name', 'date_created']]
print(oa_repo.head(5))

                               preferred_name  year_established  \
0  11th Joint Symposium on Neural Computation               NaN   
1          ACHS, Theses and Capstone Projects               NaN   
2                                        ARCH               NaN   
3                      ASU Digital Repository               NaN   
4                                      AUrora               NaN   

                        preferred_org_name      date_created  
0       California Institute of Technology    8/3/2006 10:10  
1  American College of Healthcare Sciences   7/21/2015 10:20  
2                  Northwestern University   3/19/2019 11:13  
3                 Arizona State University  11/12/2012 15:33  
4                        Auburn University   3/27/2014 11:00  


In [10]:
# 2.3 Deal with missing values

# get needed data format
oa_repo.dtypes
oa_repo['date_created'] = oa_repo['date_created'].astype('datetime64')
oa_repo['date'] = pd.DatetimeIndex(oa_repo['date_created']).year

In [11]:
# replace missing values in 'year_established' column with corresponding values in 'date' column
oa_repo.year_established.fillna(oa_repo.date, inplace=True)

print(oa_repo.head(5))

                               preferred_name  year_established  \
0  11th Joint Symposium on Neural Computation            2006.0   
1          ACHS, Theses and Capstone Projects            2015.0   
2                                        ARCH            2019.0   
3                      ASU Digital Repository            2012.0   
4                                      AUrora            2014.0   

                        preferred_org_name        date_created  date  
0       California Institute of Technology 2006-08-03 10:10:00  2006  
1  American College of Healthcare Sciences 2015-07-21 10:20:00  2015  
2                  Northwestern University 2019-03-19 11:13:00  2019  
3                 Arizona State University 2012-11-12 15:33:00  2012  
4                        Auburn University 2014-03-27 11:00:00  2014  


In [12]:
# 2.4 Add new values

# calculate the age of the OA repositories

oa_repo['age'] = 2019 - oa_repo['year_established']
print(oa_repo.head(5))

                               preferred_name  year_established  \
0  11th Joint Symposium on Neural Computation            2006.0   
1          ACHS, Theses and Capstone Projects            2015.0   
2                                        ARCH            2019.0   
3                      ASU Digital Repository            2012.0   
4                                      AUrora            2014.0   

                        preferred_org_name        date_created  date   age  
0       California Institute of Technology 2006-08-03 10:10:00  2006  13.0  
1  American College of Healthcare Sciences 2015-07-21 10:20:00  2015   4.0  
2                  Northwestern University 2019-03-19 11:13:00  2019   0.0  
3                 Arizona State University 2012-11-12 15:33:00  2012   7.0  
4                        Auburn University 2014-03-27 11:00:00  2014   5.0  


In [13]:
# only keep needed columns
oa_repo = oa_repo[['preferred_org_name', 'age']]
oa_repo.rename(columns={'preferred_org_name':'institution',
                    'age':'repo_age'}, 
                 inplace=True)
print(oa_repo.head(5))

                               institution  repo_age
0       California Institute of Technology      13.0
1  American College of Healthcare Sciences       4.0
2                  Northwestern University       0.0
3                 Arizona State University       7.0
4                        Auburn University       5.0


In [14]:
# 3.1 import the third dataset: open access mandates and policies

oap = pd.read_csv("https://raw.githubusercontent.com/caifand/DMRL_THA/master/FinalPaper/0_data_preprocessing/0_raw/roarmap.csv")

print(oap.shape)
oap.isna().sum()

(124, 53)


Unnamed: 0                     0
added_by                      12
apc_fun_url                   98
apc_funding                    0
can_deposit_be_waived          0
country                        0
country_inclusive              0
country_names                  0
creators                     123
date                         123
date_made_open                 0
date_of_deposit                0
datestamp                      0
deposit_of_item                0
dir                            0
documents                    123
embargo_hum_soc                0
embargo_sci_tech_med           0
eprint_status                  0
eprintid                       0
gold_oa_options                0
iliege_hefce_model             0
journal_article_version        0
last_revision                106
lastmod                        0
locus_of_deposit               0
making_deposit_open            0
mandate_content_types          0
maximal_embargo_waivable       0
metadata_visibility            0
open_acces

In [15]:
# 3.2 Drop needless columns

oap = oap[['policymaker_name', 'policy_adoption', 'policy_effecive']]
print(oap.head(5))

                     policymaker_name policy_adoption policy_effecive
0        Abilene Christian University         2017-05             NaN
1                   Allegheny College       2/18/2013        5/1/2013
2                     Amherst College        3/5/2013             NaN
3  Arizona State University Libraries      10/20/2010             NaN
4                       Autism Speaks             NaN       12/3/2008


In [16]:
# 3.3 Deal with missing values
# transform data types
oap['policy_effecive'] = oap['policy_effecive'].astype('datetime64')
oap['policy_adoption'] = oap['policy_adoption'].astype('datetime64')
oap.dtypes

policymaker_name            object
policy_adoption     datetime64[ns]
policy_effecive     datetime64[ns]
dtype: object

In [17]:
# replace missing values

oap['policy_effecive'].fillna(oap['policy_adoption'], inplace=True)

oap['year_effective'] = pd.DatetimeIndex(oap['policy_effecive']).year
print(oap.head(5))

                     policymaker_name policy_adoption policy_effecive  \
0        Abilene Christian University      2017-05-01      2017-05-01   
1                   Allegheny College      2013-02-18      2013-05-01   
2                     Amherst College      2013-03-05      2013-03-05   
3  Arizona State University Libraries      2010-10-20      2010-10-20   
4                       Autism Speaks             NaT      2008-12-03   

   year_effective  
0          2017.0  
1          2013.0  
2          2013.0  
3          2010.0  
4          2008.0  


In [18]:
#oap.isna().sum()
# 3.4 Calculate new variable values

oap['effective_year'] = 2019 - oap['year_effective']
print(oap.head(5))

                     policymaker_name policy_adoption policy_effecive  \
0        Abilene Christian University      2017-05-01      2017-05-01   
1                   Allegheny College      2013-02-18      2013-05-01   
2                     Amherst College      2013-03-05      2013-03-05   
3  Arizona State University Libraries      2010-10-20      2010-10-20   
4                       Autism Speaks             NaT      2008-12-03   

   year_effective  effective_year  
0          2017.0             2.0  
1          2013.0             6.0  
2          2013.0             6.0  
3          2010.0             9.0  
4          2008.0            11.0  


In [19]:
# truncate useless columns
oap = oap[['policymaker_name', 'effective_year']]
oap.rename(columns={'policymaker_name':'institution',
                    'effective_year':'policy_year'}, 
                 inplace=True)
print(oap.head(5))

                          institution  policy_year
0        Abilene Christian University          2.0
1                   Allegheny College          6.0
2                     Amherst College          6.0
3  Arizona State University Libraries          9.0
4                       Autism Speaks         11.0


In [20]:
# 4.1 merge datasets
# merge cite_rank & oa_repo
cite_repo = cite_rank.merge(oa_repo, on='institution', how='inner')
cite_repo

,institution,citation,repo_age
0,Harvard University,1,10.0
1,Harvard University,1,5.0
2,Harvard University,1,13.0
3,Massachusetts Institute of Technology,6,13.0
4,Princeton University,27,4.0
5,Columbia University,10,11.0
6,Columbia University,10,14.0
7,California Institute of Technology,31,13.0
8,California Institute of Technology,31,13.0
9,California Institute of Technology,31,13.0


In [21]:
# group by insitution and sort out the first entry within each group
cite_repo = cite_repo.groupby('institution', as_index=False)
cite_repo = cite_repo.first()

In [22]:
# merge the third dataset oap
repo_p = cite_repo.merge(oap, on='institution', how='inner')
repo_p


,institution,citation,repo_age,policy_year
0,Boston University,26,13.0,4.0
1,California Institute of Technology,31,13.0,5.0
2,Cornell University,40,13.0,14.0
3,Duke University,14,11.0,9.0
4,Emory University,63,3.0,8.0
5,Florida State University,139,11.0,3.0
6,Georgia Institute of Technology,222,13.0,6.0
7,Massachusetts Institute of Technology,6,13.0,10.0
8,Northern Illinois University,327,7.0,5.0
9,Oregon State University,187,15.0,6.0


In [23]:
#repo_p.isna().sum()
repo_p.dropna()
#print(repo_p)

,institution,citation,repo_age,policy_year
0,Boston University,26,13.0,4.0
1,California Institute of Technology,31,13.0,5.0
2,Cornell University,40,13.0,14.0
3,Duke University,14,11.0,9.0
4,Emory University,63,3.0,8.0
5,Florida State University,139,11.0,3.0
6,Georgia Institute of Technology,222,13.0,6.0
7,Massachusetts Institute of Technology,6,13.0,10.0
8,Northern Illinois University,327,7.0,5.0
9,Oregon State University,187,15.0,6.0


In [24]:
# 5.1 export intermediate data 
repo_p.to_csv("repo_p.csv", index=False)